## Prepare image

In [ ]:
import os

import cv2
import geopandas as gpd
import IPython
import matplotlib.pyplot as plt
import numpy as np
import shapely.geometry as sg

from aerial_image_detection.plot_utils import plot_obb_boxes_on_image
from aerial_image_detection.raster_utils import RasterData
from aerial_image_detection.sahi_inference import SAHIInferenceModel

def show_bgr(img):
    _, ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

RD_crs = "EPSG:28992"

classes_to_keep = [9, 10]  # ["large vehicle", "small vehicle"]

In [ ]:
full_image_path = f"../datasets/experiments/parkeren/luchtfotos/beeldmateriaal.nl/2025_115000_487000_RGB_JPEG_hrl.tif"

raster_data = RasterData(full_image_path)

print(raster_data.description())

In [ ]:
raster_data.show()

## SAHI

In [ ]:
sahi_model = SAHIInferenceModel(
    yolo_model_weights_path="../datasets/experiments/parkeren/model_weights/yolo11m-obb.pt",
    confidenc_treshold=0.2,
    image_size=1024,
    slice_height=4096,
    slice_width=4096,
    classes_to_keep=classes_to_keep,
    class_agnostic=True
)

In [ ]:
rgb_image = raster_data.as_rgb_img()

In [ ]:
sahi_result = sahi_model.predict(
    image=rgb_image,
)

In [ ]:
sahi_predictions = sahi_model.get_prediction_data(sahi_result)

obb_class = sahi_predictions["object_class"]
obb_boxes = sahi_predictions["bounding_box"]

## Visualize bounding boxes

In [ ]:
image_with_obb = plot_obb_boxes_on_image(
    raster_data.as_bgr_img(),
    obb_class,
    obb_boxes,
    single_color=(0, 180, 255),
)

show_bgr(image_with_obb)

In [ ]:
source_name = os.path.splitext(os.path.basename(full_image_path))[0]
filename = f"../datasets/experiments/parkeren/sahi_predictions_{source_name}.jpg"

_ = cv2.imwrite(filename=filename, img=image_with_obb)

## Convert bounding boxes to GPD

In [ ]:
obb_geoms = (
    gpd.GeoSeries(
        data=[sg.Polygon(coords) for coords in obb_boxes]
    )
    .affine_transform(raster_data.get_shapely_transform())
)

detections_gdf = gpd.GeoDataFrame(
    data={
        "class_id": obb_class,
        "geometry": obb_geoms
    },
    crs=RD_crs,
)

detections_gdf.insert(0, column="source_file", value=os.path.basename(full_image_path))

In [ ]:
_ = detections_gdf.plot()

## BGT

In [ ]:
bounds_poly = raster_data.get_bounds_as_polygon()

bgt_wegdeel = gpd.read_file("../datasets/experiments/parkeren/bgt/115000_487000/bgt_wegdeel.gml")
bgt_wegdeel = bgt_wegdeel[bgt_wegdeel["eindRegistratie"].isna()]

In [ ]:
bgt_voetpad = bgt_wegdeel[bgt_wegdeel.function.isin(['voetpad', 'fietspad', 'voetpad op trap', 'voetgangersgebied'])]
bgt_voetpad_area = bgt_voetpad.intersection(bounds_poly)
bgt_voetpad_area = bgt_voetpad_area[~bgt_voetpad_area.is_empty]

In [ ]:
bgt_parkeervlak = bgt_wegdeel[bgt_wegdeel.function.isin(['parkeervlak'])]
bgt_parkeervlak_area = bgt_parkeervlak.intersection(bounds_poly)
bgt_parkeervlak_area = bgt_parkeervlak_area[~bgt_parkeervlak_area.is_empty]

## Compute wrongly parked cars

In [ ]:
detections_gdf["percentage_on_sidewalk"] = (
    detections_gdf.intersection(bgt_voetpad_area.union_all(method="unary")).area 
    / detections_gdf.area
)
detections_gdf["wrongly_parked"] = detections_gdf["percentage_on_sidewalk"] >= 0.25

In [ ]:
print(f"Number of cars: {len(detections_gdf)}")
print(f"Wrongly parked: {detections_gdf['wrongly_parked'].sum()} ({(detections_gdf['wrongly_parked'].sum() / len(detections_gdf)) * 100:.1f}%)")

## Visualise all together

In [ ]:
%matplotlib inline

padding = 10

fig, ax = plt.subplots(1, figsize=(10, 10), constrained_layout=True)

[x_min, y_min, x_max, y_max] = map(int, bounds_poly.bounds)

ax.imshow(rgb_image, extent=[x_min, x_max, y_min, y_max])

bgt_voetpad_area.plot(ax=ax, color="red", alpha=0.25)
bgt_parkeervlak_area.plot(ax=ax, color="green", alpha=0.25)
detections_gdf[~detections_gdf["wrongly_parked"]].boundary.plot(ax=ax, color=np.array([255, 180, 0]) / 255)
detections_gdf[detections_gdf["wrongly_parked"]].boundary.plot(ax=ax, color=np.array([255, 0, 189]) / 255)

ax.set_xlabel('X')
ax.set_ylabel('Y')

ax.set_xticks(range(x_min, x_max+1, 100))
ax.set_xticklabels(range(x_min, x_max+1, 100))
ax.set_yticks(range(y_min, y_max+1, 100))
ax.set_yticklabels(range(y_min, y_max+1, 100))

ax.set_xlim((x_min - padding, x_max + padding))
ax.set_ylim((y_min - padding, y_max + padding))
ax.set_aspect('equal', adjustable='box')

# extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
# plt.savefig("../datasets/experiments/parkeren/full_combined.jpg", bbox_inches=extent, dpi=450)

plt.show()